<a href="https://colab.research.google.com/github/Varvara1991/Our_Poject/blob/My_branch/video_mask_r_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyyaml==5.1
# Версия torch 1.9.0
!pip install torch==1.9.0+cu102 torchvision==0.10.0+cu102 -f https://download.pytorch.org/whl/torch_stable.html

# 
# Инструкции https://detectron2.readthedocs.io/tutorials/install.html 
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu102/torch1.9/index.html
##### ПЕРЕЗАПУСТИТЕ СРЕДУ ####

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 274 kB 2.0 MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.1-cp37-cp37m-linux_x86_64.whl size=44092 sha256=d958dd91f20bd07e9c0ac2a2b44db532dc922935654f8ea8b711673373869a08
  Stored in directory: /root/.cache/pip/wheels/77/f5/10/d00a2bd30928b972790053b5de0c703ca87324f3fead0f2fd9
Successfully built pyyaml
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 6.0
    Uninstalling PyYAML-6.0:
      Successfully uninstalled PyYAML-6.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dask 2022.2.0 requires pyyaml>=5.3.1, but you have pyyaml 5.1 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Проверка версий
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
assert torch.__version__.startswith("1.9") 

1.9.0+cu102 False


In [4]:
# Устанавливаем логгер для детектрона
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# Импорты
import numpy as np
import pandas as pd
from PIL import Image
import os, json, cv2, random
from google.colab.patches import cv2_imshow

from os.path import isfile, join
from tqdm import tqdm
import copy
from google.colab.patches import cv2_imshow

from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

### Функции

In [5]:
def getFrame(sec, frame_out):
    "функция достает кадр и записывает в папку frame_out"
    vidcap.set(cv2.CAP_PROP_POS_MSEC,sec*1000)
    hasFrames,image = vidcap.read()
    if hasFrames:
        cv2.imwrite(frame_out + "/image"+str(count)+".jpg", image)   # save frame as JPG file
    return hasFrames

In [6]:
def printMaxSubSquare(M):
  R = len(M) # no. of rows in M[][]
  C = len(M[0]) # no. of columns in M[][]

  S = []
  
  # Find the maximum entry and
  # indices of maximum entry in S[][]

  max_of_s = 0 # максимальная первая
  max_i, max_j = 0, 0  # правая, нижняя координата квадрата
  

  # Слева и сверху матрицы переносим числа (O(2*R)) 
  for i in range(R): # O(R^2)
    temp = []
    for j in range(C):
      if i==0 or j==0:
        temp += M[i][j],
        if max_of_s < M[i][j]:
          max_of_s = M[i][j]
          max_i, max_j = i, j
      else:
        temp += 0,
    S += temp,

  #Заполняет соседними числами O((R-1)^2)
  for i in range(1, R):
      for j in range(1, C):
          if (M[i][j] == 1):
              S[i][j] = min(S[i][j-1], S[i-1][j],
                          S[i-1][j-1]) + 1
          else:
              S[i][j] = 0
          if max_of_s < S[i][j]:
            max_of_s = S[i][j]
            max_i = i
            max_j = j
  

  # правая, нижняя координата прямоугольника
  max_i_2 = R - 1 - np.argmax(np.array(S)[:, max_j][::-1])
  max_j_2 = C - 1 - np.argmax(S[max_i][::-1])

  h, w = max_of_s + abs(max_i_2 - max_i), max_of_s + abs(max_j_2 - max_j)
  
  if max_of_s > 1:
    max_i -= max_of_s - 1
    max_j -= max_of_s - 1

  return max_j, max_i, h, w

In [7]:
def put_mask_on(masks, image):
  for ind, mask in enumerate(masks):
    mask = mask.to('cpu').numpy()
    x, y, h, w = printMaxSubSquare(mask)

    image.paste(
        elbrus.resize((w, h)),
        (x, y))

In [8]:
!mkdir '/content/drive/MyDrive/images'

mkdir: cannot create directory ‘/content/drive/MyDrive/images’: File exists


In [9]:
#путь до видео и читаем видео

path = '/content/drive/MyDrive/IMG_7957.MOV'
vidcap = cv2.VideoCapture(path)

#путь до папки с кадрами
frame_out = '/content/drive/MyDrive/images/'
# os.mkdir(frame_out)

frameRate = 0.05 #//it will capture image in each 0.5 second

#происходит раскадровка в папку
sec = 0
count=1
success = getFrame(sec, frame_out)
while success:
    count = count + 1
    sec = sec + frameRate
    sec = round(sec, 2)
    success = getFrame(sec, frame_out)


#список имён кадров
files = [f for f in os.listdir(frame_out) if isfile(join(frame_out, f))]
#сортируем, чтобы они шли 1, 2, 3 ...
files = sorted(files, key = lambda x: int(x.split('.')[0][5:]))

In [10]:
# Используем дефолтный конфиг
cfg = get_cfg()

# Загружаем конфиг для модели YouTubeVIS 2021 R50
# model = model_zoo.get("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_1x.yaml", trained=True)
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_101_C4_3x.yaml"))

# Устанавливаем порог для детекции: если уровень доверия меньше порога, детекция не состоится
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
# Загружаем модель
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_101_C4_3x.yaml")

# Создаем объект предиктора
predictor = DefaultPredictor(cfg)

In [11]:
elbrus = "/content/drive/MyDrive/elbrus.png"
with Image.open(elbrus) as elbrus:
    elbrus.load()

In [12]:
!mkdir /content/drive/MyDrive/shots

mkdir: cannot create directory ‘/content/drive/MyDrive/shots’: File exists


In [13]:
# # Передаем в объект раскадрованные снимки
for ind, shot in tqdm(enumerate(files)):
  im = cv2.imread(frame_out+shot)
  outputs = predictor(im)
  object_list = [ind for ind, val in enumerate(outputs['instances'].pred_classes.tolist()) if val == 62]
  masks = outputs['instances'].pred_masks[object_list]
  im = frame_out+shot
  with Image.open(im) as im:
    im.load()
  put_mask_on(masks, im)
  im.save(f'/content/drive/MyDrive/shots/shot_{ind}.jpg')

0it [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
173it [07:37,  2.65s/it]


In [14]:
path_video_out = '/content/drive/MyDrive/video3.mp4'

In [15]:
fps = 20
out = cv2.VideoWriter(path_video_out,cv2.VideoWriter.fourcc(*'XVID'), fps, (int(vidcap.get(3)),int(vidcap.get(4))))
for i in range(len(files)):
    
    # writing to a image array
    img = cv2.imread(f'/content/drive/MyDrive/shots/shot_{i}.jpg')
    # cv2_imshow(img)
    # print(i)
    out.write(img)

out.release()